Importar librerías

In [ ]:
from confluent_kafka import Consumer, Producer, KafkaError
import matplotlib.pyplot as plt
from io import StringIO
import pandas as pd



In [ ]:
Productor

In [ ]:
#Carga CSV
csv_file_path = '../data/data_companies.csv'
df = pd.read_csv(csv_file_path)

producer = Producer ({'bootstrap.servers':'localhost:9092'})

for _, row in df.iterrows():
    message = f"{row['company']},{row['country']}, {row['priceLY']}, {row['priceCY']}"
    producer.produce('Companies', value = message)
    
producer.flush()



In [ ]:
Consumidor

In [ ]:
c = Consumer({'bootstrap.servers':'localhost:9092', 'group.id':'grupo1', 'auto.offset.reset':'earliest'})
c.subscribe(['Companies'])

plot_df = pd.DataFrame(columns=['company', 'country', 'priceLY', 'priceCY'])

try:
    while True:
        msg = c.poll(1.0)
        if msg is None:
            print('Ningún mensaje')
            continue
        if msg.error():
            print('Error: {}'.format(msg.error()))
            continue
        
        message_data = StringIO(msg.value().decode('utf-8'))
        row_data = pd.read_csv(message_data, header=None, names=['company', 'country', 'priceLY', 'priceCY'])
        plot_df = pd.concat([plot_df, row_data], ignore_index=True)
        
        #print('El mensaje recibido es: {}'.format(message_data))
        unique_companies = plot_df['company'].unique()
        bar_width = 0.35 
        for company in unique_companies:
            data_company = plot_df[plot_df['company'] == company]
            indexes = range(len(data_company))
            
            plt.figure(figsize=(10,6))
            plt.bar(indexes, data_company['priceLY'], width=bar_width, label='PriceLY')
            plt.bar([i + bar_width for i in indexes], data_company['priceCY'],width=bar_width, label='PriceCY')            
            plt.title(f"Price comparison for {company}")
            plt.xlabel('Index')
            plt.ylabel('Price')
            plt.xticks([i+ bar_width /2 for i in indexes], indexes)
            plt.legend()
            plt.show()
            
        
        
except KeyboardInterrupt:
    pass
finally        :
    c.close()